In [14]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run.split('_')[0]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/index_stats',
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge_alex/testrun'
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'iostat.bytes_read', 'iostat.bytes_wrtn',
       'spec.index.leaf_size_in_pages'],
      dtype='object')


,command,dataset,result.checksum,result.duration_ns,result.duration_sec,result.inner_disk_fetch,result.inner_disk_fetch_size,result.inner_total_bytes_fetched,result.num_output_keys,result.outer_disk_fetch,...,spec.load_sstable_in_mem,spec.name,spec.num_threads,spec.outer_table,spec.result_path,spec.value_size,spec.write_result_to_disk,iostat.bytes_read,iostat.bytes_wrtn,spec.index.leaf_size_in_pages
0,sponge/index_stats/build/benchmark_runner spon...,fb,0,32905777210,32.905777,1048156,8192,8586481472,199999999,781250,...,False,inlj_flatpgm2561run_0,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_f...,8,True,0,0,NaN
1,sponge/index_stats/build/benchmark_runner spon...,fb,0,45036690718,45.036691,8578348,8192,70273820496,199999999,781250,...,False,inlj_sampledflatpgm2561run_0,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_s...,8,True,0,0,NaN
2,sponge/index_stats/build/benchmark_runner spon...,fb,0,49351985372,49.351985,8578348,8192,70273820496,199999999,781250,...,False,inlj_sampledflatpgm2561run_1,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_s...,8,True,0,0,NaN
3,sponge/index_stats/build/benchmark_runner spon...,fb,0,62344506706,62.344507,781250,4096,3199999984,199999999,781250,...,False,inlj_btree2561run_1,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_b...,8,True,0,0,1.0
4,sponge/index_stats/build/benchmark_runner spon...,fb,0,48801635196,48.801635,8578348,8192,70273820496,199999999,781250,...,False,inlj_sampledflatpgm2561run_2,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_s...,8,True,0,0,NaN
5,sponge/index_stats/build/benchmark_runner spon...,fb,0,178038561874,178.038562,191066017,4096,782606218208,199999999,781250,...,False,inlj_radixspline2561run_2,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_r...,8,True,0,0,NaN
6,sponge/index_stats/build/benchmark_runner spon...,fb,0,37193262219,37.193262,1048156,8192,8586481472,199999999,781250,...,False,inlj_flatpgm2561run_2,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_f...,8,True,0,0,NaN
7,sponge/index_stats/build/benchmark_runner spon...,fb,0,53578359859,53.578360,781250,4096,3199999984,199999999,781250,...,False,inlj_btree2561run_0,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_b...,8,True,0,0,1.0
8,sponge/index_stats/build/benchmark_runner spon...,fb,0,38622879907,38.622880,1048156,8192,8586481472,199999999,781250,...,False,inlj_flatpgm2561run_1,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_f...,8,True,0,0,NaN
9,sponge/index_stats/build/benchmark_runner spon...,fb,0,193167742377,193.167742,191066017,4096,782606218208,199999999,781250,...,False,inlj_radixspline2561run_1,1,sponge/index_stats/fb_threads=1/inputs/input1,sponge/index_stats/fb_threads=1/outputs/inlj_r...,8,True,0,0,NaN


In [15]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    elif "alex" in x:
        return "ALEX"
    elif "RMI" in x:
        return "RMI"
    return "NA"

df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["inner_disk_fetch"] = df["result.inner_disk_fetch"]
df['result_num_keys'] = df['result.num_output_keys']
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["epsilon"] = df["spec.index.epsilon"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['disk_read_gb'] = df['iostat.bytes_read']/1e9
df['disk_wrtn_gb'] = df['iostat.bytes_wrtn']/1e9
#df['cdisk_read_gb'] = df['cgroup_iostat.bytes_read']/1e9
#df['cdisk_wrtn_gb'] = df['cgroup_iostat.bytes_wrtn']/1e9

df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")



┌──────────────────────┬─────────┬─────────────────┬───┬────────────┬──────────────┬──────────────┐
│       command        │ dataset │ result.checksum │ … │ index_type │ disk_read_gb │ disk_wrtn_gb │
│       varchar        │ varchar │      int64      │   │  varchar   │    double    │    double    │
├──────────────────────┼─────────┼─────────────────┼───┼────────────┼──────────────┼──────────────┤
│ sponge/index_stats…  │ fb      │               0 │ … │ PGM        │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb      │               0 │ … │ PGM        │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb      │               0 │ … │ PGM        │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb      │               0 │ … │ BTREE      │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb      │               0 │ … │ PGM        │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb      │               0 │ … │ NA         │          0.0 │          0.0 │


In [17]:
tmpdf = duckdb.sql("SELECT RATIO,ALGO,AVG(DURATION_SEC) FROM df GROUP BY ratio,algo ORDER BY ratio,algo").df()
display(tmpdf)

,ratio,algo,avg(DURATION_SEC)
0,1,alex_inlj,603.018063
1,1,inlj_btree256,54.399723
2,1,inlj_flatpgm256,36.240640
3,1,inlj_radixspline256,189.289614
4,1,inlj_sampledflatpgm256,47.730104
